In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import datetime as dt
from tqdm import tqdm
import json
from thefuzz import fuzz, process
from tenacity import retry
from tenacity import stop_after_attempt, wait_fixed, Retrying

from nba_api.stats.endpoints import leaguedashteamstats, playerawards

pd.options.mode.chained_assignment =  None

data_DIR = "../../data/misc/"

In [ ]:
dfp = pd.read_csv("../../data/NBA_players_database.csv")
dfp = dfp.sort_values(["To","pID"],ascending=[False,True]).reset_index(drop=True)

In [ ]:
pID_dict = dict(zip(dfp["Name"],dfp["pID"]))
player_dict = dict(zip(dfp["pID"],dfp["Name"]))
pID_list = dfp["pID"].to_list()

In [ ]:
@retry(stop=stop_after_attempt(5),wait=wait_fixed(1))
def get_player_award(id):
    time.sleep(1)
    stats = playerawards.PlayerAwards(player_id=id)
    df21 = stats.get_data_frames()[0]
    return df21

In [ ]:
dfa = []
eID = []
for id in tqdm(pID_list):
    try:
        df21 = get_player_award(id)
        dfa.append(df21)
    except Exception as e:
        eID.append(id)
        print(e)
df3 = pd.concat(dfa)

In [ ]:
df3.to_parquet(data_DIR+"NBA_Awards_2024.parquet")

In [ ]:
fgdgf

In [ ]:
def get_missing_pId(player,player_dict):
    pId = process.extract(player,player_dict,limit=1, scorer=fuzz.partial_ratio)[0][2]
    return pId

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_NBA_All-Stars"
df = pd.read_html(url)

In [ ]:
df1 = df[1]
df1["Player"] = df1["Player"].str.replace(r"*", "")
df1["Player"] = df1["Player"].str.replace(r"^", "")
df1["Player"] = df1["Player"].str.replace(r"[a]", "")
df1["Player"] = df1["Player"].str.replace(r"[b]", "")
df1["Player"] = df1["Player"].str.replace(r"[c]", "")
df1["Player"] = df1["Player"].str.replace(r"[d]", "")
df1["Player"] = df1["Player"].str.replace(r"†", "")
df1["Player"] = df1["Player"].str.replace(r"§", "")
df1["Player"] = df1["Player"].str.replace("Penny Hardaway", "Anfernee Hardaway")
df1["Player"] = df1["Player"].str.replace("Steve Smith", "Steven Smith")
df1["pID"] = df1["Player"].map(pID_dict)
df1.loc[df1["pID"].isna(),"pID"] = df1.loc[df1["pID"].isna(),"Player"].apply(lambda x: get_missing_pId(x, player_dict))
df1["pID"] = df1["pID"].astype(int)
# df1["Player2"] = df1["pID"].map(player_dict)
# df1["Verify"]  = df1["Player2"] != df1["Player"]
pIDs = df1["pID"].unique()

In [ ]:
dfa = []
for id in tqdm(pIDs):
    try:
        stats = playerawards.PlayerAwards(player_id=id)
        df21 = stats.get_data_frames()[0]
        dfa.append(df21)
    except Exception as e:
        print(e)
    time.sleep(1)
df2 = pd.concat(dfa)